<a href="https://colab.research.google.com/github/eslywadan/bert/blob/master/%E3%80%8Cmt5_ipynb%E3%80%8D%E7%9A%84%E5%89%AF%E6%9C%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# change runtime to use gpu

In [2]:
!pip install bert4keras --quiet
!pip install gsutil --quiet
!pip install tensorflow-gpu==1.14 --quiet
!pip install rouge --quiet
!pip install sentencepiece --quiet

     |████████████████████████████████| 53 kB 2.6 MB/s 
     |████████████████████████████████| 377 kB 53.0 MB/s 
     |████████████████████████████████| 50 kB 6.3 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.2+zzzcolab20220719082949 requires keras<2.9,>=2.8.0rc0, but you have keras 2.3.1 which is incompatible.
     |████████████████████████████████| 3.0 MB 14.4 MB/s 
     |████████████████████████████████| 135 kB 11.5 MB/s 
     |████████████████████████████████| 96 kB 2.0 MB/s 
     |████████████████████████████████| 57 kB 5.2 MB/s 
     |████████████████████████████████| 169 kB 49.6 MB/s 
     |████████████████████████████████| 1.4 MB 14.8 MB/s 
     |████████████████████████████████| 4.0 MB 60.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the sourc

In [3]:
import subprocess

In [4]:
subprocess.run(["gsutil", "cp",  "-r",  "gs://t5-data/pretrained_models/mt5/small",  "."])

CompletedProcess(args=['gsutil', 'cp', '-r', 'gs://t5-data/pretrained_models/mt5/small', '.'], returncode=0)

In [ ]:
subprocess.run(["gsutil",  "cp", "-r", "gs://t5-data/vocabs/mc4.250000.100extra/sentencepiece.model", "."])

CompletedProcess(args=['gsutil', 'cp', '-r', 'gs://t5-data/vocabs/mc4.250000.100extra/sentencepiece.model', '.'], returncode=0)

In [ ]:
# !wget https://github.com/bojone/t5_in_bert4keras/blob/main/tokenizer/sentencepiece_cn.model
# !wget https://github.com/bojone/t5_in_bert4keras/blob/main/tokenizer/sentencepiece_cn_keep_tokens.json

--2022-10-03 07:52:29--  https://github.com/bojone/t5_in_bert4keras/blob/main/tokenizer/sentencepiece_cn.model
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘sentencepiece_cn.model.1’

sentencepiece_cn.mo     [ <=>                ] 139.93K  --.-KB/s    in 0.01s   

2022-10-03 07:52:29 (11.6 MB/s) - ‘sentencepiece_cn.model.1’ saved [143293]



In [ ]:
import json
import numpy as np
from tqdm.auto import tqdm

from bert4keras.backend import keras, K
from bert4keras.layers import Loss
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import SpTokenizer
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, open
from bert4keras.snippets import DataGenerator, AutoRegressiveDecoder
from keras.models import Model
from rouge import Rouge  # pip install rouge
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
def load_data(filename):
    D = []
    with open(filename, encoding='utf-8') as f:
        for i, l in enumerate(f):
            if i == 0:
                continue
            title, abstract, keywords, discipline, category = l.strip().split('\t')
            D.append((title, abstract))
    return D

In [ ]:
class data_generator(DataGenerator):
    """数据生成器
    """
    def __iter__(self, random=False):
        batch_c_token_ids, batch_t_token_ids = [], []
        for is_end, (title, content) in self.sample(random):
            c_token_ids, _ = tokenizer.encode(content, maxlen=max_c_len)
            t_token_ids, _ = tokenizer.encode(title, maxlen=max_t_len)
            batch_c_token_ids.append(c_token_ids)
            batch_t_token_ids.append([0] + t_token_ids)
            if len(batch_c_token_ids) == self.batch_size or is_end:
                batch_c_token_ids = sequence_padding(batch_c_token_ids)
                batch_t_token_ids = sequence_padding(batch_t_token_ids)
                yield [batch_c_token_ids, batch_t_token_ids], None
                batch_c_token_ids, batch_t_token_ids = [], []

In [ ]:
class CrossEntropy(Loss):
    """交叉熵作为loss，并mask掉输入部分
    """
    def compute_loss(self, inputs, mask=None):
        y_true, y_pred = inputs
        y_true = y_true[:, 1:]  # 目标token_ids
        y_mask = K.cast(mask[1], K.floatx())[:, :-1]  # 解码器自带mask
        y_pred = y_pred[:, :-1]  # 预测序列，错开一位
        loss = K.sparse_categorical_crossentropy(y_true, y_pred)
        loss = K.sum(loss * y_mask) / K.sum(y_mask)
        return loss

In [ ]:
config_path = './mt5_small_config.json'
checkpoint_path = './small/model.ckpt-1000000'
spm_path = './sentencepiece.model'

In [ ]:
mt5_small_cfg = {
  "hidden_dropout_prob": 0.1,
  "hidden_size": 512,
  "initializer_range": 0.02,
  "intermediate_size": 1024,
  "num_attention_heads": 6,
  "attention_head_size": 64,
  "num_hidden_layers": 8,
  "vocab_size": 250112,
  "hidden_act": ["gelu", "linear"]
}
with open(config_path, 'w') as f:
    json.dump(mt5_small_cfg, f)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# https://github.com/ydli-ai/CSL
# https://drive.google.com/drive/u/3/folders/1f1D6kk4dmfJv5iFQ2jLkxAykLiou6xcz
# add to drive

In [ ]:
train_data = load_data('/content/drive/MyDrive/Colab Notebooks/csl_40k.tsv')
train_data[0][0], train_data[0][1]

('分布式系统中的检查点算法',
 '检查点能够保存和恢复程序的运行状态.它在进程迁移、容错、卷回调试等领域都有重要的应用.本文对分布式系统中的检查点算法进行了详细的分类评述.检查点算法可分为单进程和分布式程序检查点算法,分布式程序检查点算法又可分为异步检查点算法和一致检查点算法.同时本文系统介绍了改进检查点算法性能的典型方法.这些改进算法主要采用两个策略来减少算法的开销与延迟:一是减少检查点文件中需要存储的信息量,如增量算法等;二是提高检查点操作与目标程序运行的并行性,如主存算法等.最后,文章讨论了目前检查点算法的局限性和进一步的工作.')

In [ ]:
tokenizer = SpTokenizer(spm_path, token_start=None, token_end='&lt;/s&gt;')

In [ ]:
t5 = build_transformer_model(
    config_path=config_path,
    checkpoint_path=checkpoint_path,
    model='mt5.1.1',
    return_keras_model=False,
    name='T5',
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [ ]:
encoder = t5.encoder
decoder = t5.decoder
model = t5.model
model.summary()

output = CrossEntropy(1)([model.inputs[1], model.outputs[0]])

model = Model(model.inputs, output)
model.compile(optimizer=Adam(2e-4))

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder-Input-Token (InputLayer (None, None)         0                                            
__________________________________________________________________________________________________
Encoder-Input-Token (InputLayer (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 512)    128057344   Encoder-Input-Token[0][0]        
                                                                 Decoder-Input-Token[0][0]        
__________________________________________________________________________________________________
Encoder-Embedding-Dropout (Drop (None, None, 512)    0           Embedding-Token[0][0]      

/usr/local/lib/python3.7/dist-packages/keras/engine/training_utils.py:819: UserWarning: Output cross_entropy_1 missing from loss dictionary. We assume this was done on purpose. The fit and evaluate APIs will not be expecting any data to be passed to cross_entropy_1.
  'be expecting any data to be passed to {0}.'.format(name))


In [ ]:
max_c_len = 256
max_t_len = 32
batch_size = 16
epochs = 40

In [ ]:
train_generator = data_generator(train_data, batch_size)

In [ ]:
model.fit(
  train_generator.forfit(),
  steps_per_epoch=len(train_generator),
  epochs=epochs,
)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_util.py:90: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 128057344 elements. This may consume a large amount of memory.
  num_elements)



Epoch 1/40
2500/2500 [==============================] - 1678s 671ms/step - loss: 4.0113
Epoch 2/40
2473/2500 [============================>.] - ETA: 18s - loss: 2.3497